# Converter o GRIB em um Ctl

In [1]:
import pygrib 
import numpy as np

BASE_PATH = r'./grib1/'
GRIB_FILE = 'GPOSNMC20170906122017090618P.fct.TQ0666L064.grb'
# GRIB_PATH = r'GPOSNMC20170202122017020312P.grib2'

grib = pygrib.open(BASE_PATH + GRIB_FILE)

# for g in grib:
#     print(g)

In [2]:
# Obter o nome das variáveis
def get_names(grib):
    grib.seek(0)
    names = []
    for g in grib:
        names.append(g['name'])
    return names
    
# Retornar o nome das variáveis contadas
def count_vars(grib):
    grib.seek(0)
    var_counted = []
    # Pegar o nome das variáveis
    names = get_names(grib)
    for n in names:
        if not var_counted or var_counted[-1][0] != n:
            var_counted.append((n, names.count(n)))
    
    return var_counted

# Retorna as variáveis e as unidades
def get_names_units(grib):
    grib.seek(0)
    names_units =[]
    
    for num, g in enumerate(grib):
        if not names_units or names_units[-1][0] != g['name']:
            names_units.append((g["name"], g["units"]))
            
    return names_units


# counted_vars = count_vars(grib)
# names_units = get_names_units(grib)

names = get_names(grib)
# print(names)

In [3]:
msg = grib[1]

# msg['latitudes'] # isso está muito estranho

In [4]:
print(len(msg['distinctLatitudes']))
# msg = None

1000


In [5]:
from datetime import datetime

header = []
header.append(GRIB_FILE)
# msg = grib[1]

# Checar se a mensagem é classificada em nível de pressão
def has_levels(type_level):
    if type_level != 'isobaricInhPa': 
        return False
    else: 
        return True

# Obter os levels de pressão
def get_levels(grib):
    grib.seek(0)
    for msg in grib:
        if has_levels(msg['typeOfLevel']):
            selected = grib.select(name=msg['name'], typeOfLevel=msg['typeOfLevel'])
            levels = [grb['level'] for grb in selected]
            return levels

        
# Obter o início das latitudes
def get_start_lat(msg):
    lat = msg['latitudeOfFirstGridPointInDegrees']
    return lat


# Obter o início das longitudes
def get_start_lon(msg):
    lon = msg['longitudeOfFirstGridPointInDegrees']
    return lon

# Incremento das latitudes?? em graus
def get_i_increment(msg):
    return msg['iDirectionIncrementInDegrees']

# Incremento das longitudes?? em graus
def get_j_increment(msg):
    return msg['jDirectionIncrementInDegrees']


# Contar as latitudes
def count_lats(msg):
    return len(msg['distinctLatitudes'])

# Contar as longitudes
def count_lons(msg):
    return len(msg['distinctLongitudes'])


# Obter o dataDate do arquivo
def get_datadate(msg):
    # calendário para substituir pelo acrônimo do mês
    calendar = {1:'JAN', 2:'FEB', 3:'MAR', 4:'APR', 5:'MAY', 6:'JUN',
           7:'JUL', 8:'AUG', 9:'SEP', 10:'OCT', 11:'NOV', 12:'DEC'}
    
    # essa data precisa ser convertida para o formato de dd-mm-yy
    data_date = str(msg['dataDate'])
    date_time = datetime.strptime(data_date, '%Y%m%d')
    
    # fazer a conversão
    data_date = date_time.strftime('%d-%m-%Y')
    data_date = data_date.split('-')
    data_date[1] = calendar[int(data_date[1])]
    
    # appendar em uma string para retornar
    test = ''
    # [test += str(d) for d in data_date]
    for d in data_date:
        test += str(d)
        
    hour_z = str(msg['validityTime'])[0:2]
    hr = msg['endStep']
    date = '%sZ%s  %shr' % (hour_z, test, hr)
    return date


xdef = [count_lons(msg), get_start_lon(msg), get_i_increment(msg)]
ydef = [count_lats(msg), get_start_lat(msg), get_j_increment(msg)]
tdef = [get_datadate(msg)]

levels = get_levels(grib)
# print(levels)
# print(len(levels))

zdef = [len(levels), levels]

# header.append('xdef %s' % xdef)
# header.append('ydef %s' % ydef)
# header.append('tdef %s' % tdef)
# header.append('zdef %s' % zdef)

header.append(xdef)
header.append(ydef)
header.append(tdef)
header.append(zdef)


In [6]:
header

['GPOSNMC20170906122017090618P.fct.TQ0666L064.grb',
 [2000, 0.0, 0.18],
 [1000, -90.0, 0.18],
 ['18Z06SEP2017  6hr'],
 [33,
  [1020,
   1000,
   975,
   950,
   925,
   900,
   875,
   850,
   825,
   800,
   775,
   750,
   725,
   700,
   675,
   650,
   600,
   550,
   500,
   450,
   400,
   350,
   300,
   250,
   200,
   150,
   100,
   70,
   50,
   30,
   20,
   10,
   3]]]

In [7]:
import re

CTL_FILE = open(r'./ctl/GPOSNMC20170906122017090618P.fct.TQ0666L064.ctl', 'r')
ctl_content = CTL_FILE.read()

# função para capturar o Titulo do arquivo CTL utilizando Expressão Regular
# pois o título é gerado automaticamente pelo software
def get_title(ctl_content):
    m = re.search('title .*', ctl_content)
    return str(m.group())

def get_dtype(ctl_content):
    m = re.search('dtype .*', ctl_content)
    return str(m.group())

title = get_title(ctl_content)
dtype = get_dtype(ctl_content)
# dtype

In [8]:
from datetime import datetime





# Obter o dataDate do arquivo
def get_datadate(msg):
    # calendário para substituir pelo acrônimo do mês
    calendar = {1:'JAN', 2:'FEB', 3:'MAR', 4:'APR', 5:'MAY', 6:'JUN',
           7:'JUL', 8:'AUG', 9:'SEP', 10:'OCT', 11:'NOV', 12:'DEC'}
    
    # essa data precisa ser convertida para o formato de dd-mm-yy
    data_date = str(msg['dataDate'])
    date_time = datetime.strptime(data_date, '%Y%m%d')
    
    # fazer a conversão
    data_date = date_time.strftime('%d-%m-%Y')
    data_date = data_date.split('-')
    data_date[1] = calendar[int(data_date[1])]
    
    test = ''
    # [test += str(d) for d in data_date]
    
    for d in data_date:
        test += str(d)
        
    
    
    hour_z = str(msg['validityTime'])[0:2]
    hr = msg['endStep']
    date = '%sZ%s  %shr' % (hour_z, test, hr)
    return date

# str(msg['validityTime'])[0:2]
get_datadate(msg)

'18Z06SEP2017  6hr'

Essa célula foi escrita para formatar o header do arquivo, conhecido como Header

In [9]:
def get_specific_height(msg):
    if has_levels(msg['typeOfLevel']):
        return 0
    else:
        return msg['level']

grib.seek(0)
vars_grib = []

# Adicionar as variáveis em uma lista
for msg in grib:
    if not vars_grib or vars_grib[-1][5] != msg['name']:
        vars_grib.append([msg['shortName'], names.count(msg['name']),
                         msg['indicatorOfParameter'], 'LTYPE', get_specific_height(msg),
                         msg['name'], '(%s)' % msg['units'] ])
        
for v in vars_grib:
    print(v)

[u'topo', 1, 132, 'LTYPE', 0, u'Topography', u'(m)']
[u'lsmk', 1, 81, 'LTYPE', 0, u'Land sea mask', u'((0 - 1))']
[u'pslc', 1, 135, 'LTYPE', 0, u'Surface pressure', u'(hPa)']
[u'uves', 1, 192, 'LTYPE', 0, u'Surface zonal wind (u)', u'(m s**-1)']
[u'uvel', 33, 33, 'LTYPE', 0, u'Zonal wind (u)', u'(m s**-1)']
[u'vves', 1, 194, 'LTYPE', 0, u'Surface meridional wind (v)', u'(m s**-1)']
[u'vvel', 33, 34, 'LTYPE', 0, u'Meridional wind (v)', u'(m s**-1)']
[u'omeg', 33, 39, 'LTYPE', 0, u'Omega', u'(Pa s**-1)']
[u'vort', 33, 43, 'LTYPE', 0, u'Vorticity', u'(s**-1)']
[u'fcor', 33, 35, 'LTYPE', 0, u'Stream function', u'(m**2 s**-1)']
[u'potv', 33, 36, 'LTYPE', 0, u'Velocity potential', u'(m**2 s**-1)']
[u'zgeo', 33, 7, 'LTYPE', 0, u'Geopotential height', u'(gpm)']
[u'psnm', 1, 2, 'LTYPE', 0, u'Pressure reduced to msl', u'(hPa)']
[u'tems', 1, 188, 'LTYPE', 0, u'Surface absolute temperature', u'(K)']
[u'temp', 33, 11, 'LTYPE', 0, u'Absolute temperature', u'(K)']
[u'umrs', 1, 226, 'LTYPE', 0, u'Surf

In [10]:

grib_ctl = open('grib_ctl.txt', 'wa')


def write_title(header_list, grib_ctl):
#     grib_ctl.seek(0)
    grib_ctl.write('dset ^%s' % GRIB_FILE)
    grib_ctl.write('\n')
    grib_ctl.write('*')
    grib_ctl.write('\n')
    grib_ctl.write('index ^{0}{1}'.format(GRIB_FILE[:-3], 'idx'))
    grib_ctl.write('\n')
    grib_ctl.write('*')
    grib_ctl.write('\n')
    grib_ctl.write(title)
    grib_ctl.write('\n')
    grib_ctl.write('*')
    grib_ctl.write('\n')
    grib_ctl.write(dtype)
    grib_ctl.write('\n')
    grib_ctl.write('*')
    grib_ctl.write('\n')

# escrever as definições da coordenada das longitudes
def write_xdef(header_list, grib_ctl):
    grib_ctl.write('options yrev')
    grib_ctl.write('\n')
    grib_ctl.write('*')
    grib_ctl.write('\n')
    # pegar a posição da lista do header para fazer a formatação
    grib_ctl.write('xdef{0}{1} linear  {2}  {3}'.format(2*' ', header[1][0],
                                                   header[1][1], header[1][2]))
    grib_ctl.write('\n')
    

def write_ydef(header_list, grib_ctl):
    grib_ctl.write('ydef{0}{1} linear  {2}  {3}'.format(2*' ', header[2][0],
                                                   header[2][1], header[2][2]))
    grib_ctl.write('\n')
            

def write_tdef(header_list, grib_ctl):
    grib_ctl.write('tdef{0}{1} linear  {2}  {3}'.format(2*' ', len(header[3]),
                                               header[3][0], header[3]))
    grib_ctl.write('\n')
    grib_ctl.write('*')
    grib_ctl.write('\n')
    
    
def write_zdef(header_list, grib_ctl):
    grib_ctl.write('zdef{0}{1} levels {2}'.format(2 * ' ',
                                                 header[4][0],
                                                 header[4][1]))
    grib_ctl.write('\n')

    
# escrever a quantidade de variaveis
# são apenas as variáveis sem detalhamento
def write_vars_number(vars_grib, grib_ctl):
    grib_ctl.write('\n')
    grib_ctl.write('vars{0}{1}'.format(4*' ', len(vars_grib)))
    grib_ctl.write('\n')
    

def write_vars(vars_grib, grib_ctl):
    for var in vars_grib:
        grib_ctl.write('{0}  {1}  {2},  {3},  {4}  ** {5}{6}{7}{8}'
                       .format(var[0].upper(), var[1], var[2], var[3], var[4],
                              3 * ' ', var[5], 4 * ' ', var[6]))
        grib_ctl.write('\n')
    
    grib_ctl.write('endvars')
                    


In [11]:
write_title(header, grib_ctl)
write_xdef(header, grib_ctl)
write_ydef(header, grib_ctl)
write_tdef(header, grib_ctl)
write_zdef(header, grib_ctl)

write_vars_number(vars_grib, grib_ctl)
write_vars(vars_grib, grib_ctl)

In [ ]:
thefile = open('test.txt', 'w')

# def write_header(file_to_write, header):
#     for h in header:
#         file_to_write.write(str(h))
#         file_to_write.write('\n')

# def write_vars(file_to_write, vars_to_write):
# #     thefile.write(str(v))
# #     thefile.write('\n')(file_to_write, vars_to_write):
#     for v in vars_to_write:
#         file_to_write.write(str(v))
#         file_to_write.write('\n')
        
        
# Escrever os resultados em um arquivo de text
for h in header:
    thefile.write(str(h))
    thefile.write('\n')

# write_header(thefile, header)

thefile.write('\n')
thefile.write('vars %s' %len(vars_grib))
thefile.write('\n')

# write_vars(thefile, vars_grib)

for v in vars_grib:
    thefile.write(str(v))
    thefile.write('\n')

thefile.write('endvars')

grib.close()

# vars_grib[-1]

In [ ]:
for msg in grib[0:4]:
    try:
        print(msg)
        for k in msg.keys():
            print("%s -> %s" % (msg[k], k))
        print("-----------------------------------")
    except RuntimeError:
        print("Runtime Error / Key not found")

In [ ]:
msg = grib[3]
for k in msg.keys(): print(k)
print('---------------------------------------------------------------')
print(msg['bottomLevel'])